In [1]:
from scipy import misc
import scipy
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import os, sys
import glob
import pandas as pd
import csv
from data.img.ArtDataLoader import ArtData

In [2]:
from scipy import misc
art_data = ArtData()
art_data.load_images()
art_data.load_metadata()

ArtData Initialized!
All images loaded!


In [3]:
print(art_data.train_images.keys())
print(art_data.alpha_pairs['same'][0][95])
print(art_data.beta_pairs['same'][0][95])

print(art_data.alpha_pairs['diff'][0][95])
print(art_data.beta_pairs['diff'][0][95])

dict_keys(['1440-impressionist-16x16.jpg', '0321-cubist-16x16.jpg', '2523-impressionist-16x16.jpg', '1988-impressionist-16x16.jpg', '1232-cubist-16x16.jpg', '1792-impressionist-16x16.jpg', '0373-cubist-16x16.jpg', '1455-impressionist-16x16.jpg', '0554-cubist-16x16.jpg', '1712-impressionist-16x16.jpg', '0701-cubist-16x16.jpg', '1650-impressionist-16x16.jpg', '0538-cubist-16x16.jpg', '1117-cubist-16x16.jpg', '0668-cubist-16x16.jpg', '1921-impressionist-16x16.jpg', '0194-cubist-16x16.jpg', '0285-cubist-16x16.jpg', '0512-cubist-16x16.jpg', '1721-impressionist-16x16.jpg', '0397-cubist-16x16.jpg', '1882-impressionist-16x16.jpg', '2112-impressionist-16x16.jpg', '2528-impressionist-16x16.jpg', '1493-impressionist-16x16.jpg', '1891-impressionist-16x16.jpg', '2240-impressionist-16x16.jpg', '1970-impressionist-16x16.jpg', '2002-impressionist-16x16.jpg', '1475-impressionist-16x16.jpg', '1554-impressionist-16x16.jpg', '2282-impressionist-16x16.jpg', '1899-impressionist-16x16.jpg', '0139-cubist-16x1

In [7]:
print(art_data.train_images['0001-cubist-16x16.jpg'].shape)

(16, 16, 3)
